# Weather Features

*Per hour*

- Temperature
- Wind Speed
- Precipitation Total

In [1]:
# package imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
weather_airport = pd.read_csv('../../data/raw/1399060.csv', usecols=[5, 6, 10, 17, 24], dtype='object')
weather_airport['DATE'] = pd.to_datetime(weather_airport['DATE'])
weather_airport.set_index('DATE', inplace=True)
weather_airport['temp'] = pd.to_numeric(weather_airport.HOURLYDRYBULBTEMPF.str.replace('[^\d]', ''), errors='coerce')
weather_airport['wind_speed'] = weather_airport.HOURLYWindSpeed.str.replace('[a-zA-Z]', '').astype('float')
weather_airport.HOURLYPrecip.replace({'T': '0.00'}, inplace=True)
weather_airport['precipitation'] = weather_airport.HOURLYPrecip.str.replace('[a-zA-Z]', '').astype('float')

# remove 'Summary of Day' Report
weather_airport = weather_airport[weather_airport['REPORTTPYE'] != 'SOD']

weather_airport = weather_airport[['temp', 'wind_speed', 'precipitation']]

weather_airport_by_hour = weather_airport.resample('1h').max()

weather_airport_by_hour['precipitation'] = weather_airport_by_hour.precipitation.fillna(0)
weather_airport_by_hour['temp'] = weather_airport_by_hour.temp.interpolate()
weather_airport_by_hour['wind_speed'] = weather_airport_by_hour.wind_speed.interpolate()

In [3]:
weather_airport_by_hour.to_pickle('../../data/interim/features/weather.pickle')